### Format text files into a CSV file  
folders: target_dir/category/text_files    

#### Import libraries  

In [1]:
import os, glob
import pandas as pd
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter, LowerCaseFilter

#### Parameters  

In [2]:
wakati = False
target_dir = 'text_fruits'
csv_out = 'fruits.csv'

#### Setup for wakati  

In [3]:
token_filters = [ POSStopFilter(['助詞','助動詞']),
                  LowerCaseFilter(),
                ]
tokenizer = Tokenizer()
analyzer = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

#### Read text files  

In [4]:
file_pat = '*.txt'
dir_pat = os.path.join(target_dir, '*')

text = []
for d in glob.glob(dir_pat):  # for-loop of directories in target_dir
    if not os.path.isdir(d): continue  # skip if d is not a directory
    category = os.path.basename(d)  # use directory name as the category name
    full_pat = os.path.join(d, file_pat)  # generate full path name for text files
    n_files = 0  # counter for number of files
    for file in glob.glob(full_pat):  # for-loop of text files
        n_files += 1
        with open(file, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines()  # if you want to omit newline character
            body = ' '.join(lines[2:])  # 1st line: URL, 2nd line: Date, so skip these lines
            if wakati:  # add space between words (wakati-gaki)
                body = ' '.join([t.base_form for t in analyzer.analyze(body)])
            text.append([category, body])
    print(n_files, 'files read for', category)

1 files read for orange
1 files read for grape
1 files read for apple


In [5]:
df = pd.DataFrame(text, columns=['category', 'text'])
print(df.shape)
print(df.info())
display(df.head())
display(df.tail())

(3, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  3 non-null      object
 1   text      3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes
None


,category,text
0,orange,
1,grape,
2,apple,


,category,text
0,orange,
1,grape,
2,apple,


In [6]:
df.to_csv(csv_out, index=False)